In [1]:
from source.project2.training_scripts import predict_wav_to_vec
predictions, filenames = predict_wav_to_vec()

c:\Users\gosia\Desktop\studia\mgr_sem_1\DL\python2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
c:\Users\gosia\Desktop\studia\mgr_sem_1\DL\python2\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:748: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h

In [2]:
import pandas as pd

res = {'fname': 'label'}
res.update({k: v for k, v in zip(filenames,predictions)})
res = pd.Series(res)
res = res.apply(lambda x : 'unknown' if x in UNKNOWN_DIRS else x)
res.to_csv("wav_2_vec.csv", header=None)

In [18]:
import editdistance

# Edit distance == 0

In [23]:
data = pd.read_csv("wav_2_vec.csv",index_col=0)
data.reset_index(inplace = True)
data['fname'] = data['fname'].apply(lambda x: x.split('\\')[-1])
mapping = [ 'yes',  'no',  'up',  'down',  'left', 'right',
                'on',  'off', 'stop', 'go', 'silence']
UNKNOWN_DIRS = ["bed", "bird", "cat", "dog", "eight", "five", "four", "happy", "house", "marvin", "nine", "one", "seven", "sheila", "six", "three", "tree", "two", "wow", "zero"]
data.set_index('fname').to_csv('wav_2_vec_3.csv')

# Edit distance == 1

In [62]:
def change_to_the_closest(label,mapping):
    for word in mapping:
        try:
            _ = label.split(' ')
            if word in _:
                return word
            
            if any([editdistance.eval(x, word) ==1 for x in _]):
                return word
        except Exception as e:
            return 'silence'

    return 'unknown'


In [63]:
def check_if_exists(label,mapping,existing):
    if label in existing:
        return label
    return change_to_the_closest(label,mapping)

In [64]:
existing = [ 'yes',  'no',  'up',  'down',  'left', 'right', 'on',  'off', 'stop', 'go',"unknown",'silence']
mapping = [ 'yes',  'no',  'up',  'down',  'left', 'right',
                'on',  'off', 'stop', 'go']
data['label'] = data['label'].apply(lambda x: check_if_exists(x,mapping,existing))

In [65]:
data.set_index('fname').to_csv('wav_2_vec_4.csv')